In [1]:
%run ConsultaResultado.py

In [2]:
cn = ConsultasResultado()

In [3]:
cn.ConsultaAlcaldias()

,Alcaldias
0,Xochimilco
1,Tlalpan
2,Coyoacán
3,Iztapalapa
4,Álvaro Obregón
5,Benito Juárez
6,Iztacalco
7,Cuauhtémoc
8,Miguel Hidalgo
9,Venustiano Carranza


In [4]:
cn.ConsultaUnidades()

,vehicleId
0,170
1,177
2,1286
3,1302
4,361
...,...
200,1238
201,1024
202,10002
203,1232


In [5]:
cn.ConsultaCoordenadasUnidad(170)

,positionLatitude,positionLongitude
0,19.317499,-99.187798


In [6]:
cn.ConsultaunidadesByAlcaldia('Tlalpan')

,vehicleId
0,177
1,531
2,10016
3,1257
4,1177
5,533
6,1254
